<a href="https://colab.research.google.com/github/ajaysinghrathore1/learn/blob/main/summerization_using_free_llm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain faiss-cpu transformers sentence-transformers huggingface_hub pypdf
#

In [2]:
!pip install -U langchain-community

In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline

from transformers import pipeline

# 1. Load PDF and split into chunks
pdf_path = "/content/sample_data/20220202_alphabet_10K.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=100)
docs = splitter.split_documents(documents)

# 2. Create Embeddings using sentence-transformers (or HuggingFaceEmbeddings)
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 3. Vector store using FAISS
vectorstore = FAISS.from_documents(docs, embedding_model)

# 4. Create Retriever
retriever = vectorstore.as_retriever()

# 5. Load summarization model from HuggingFace
summarizer_pipe = pipeline("summarization", model="google/pegasus-xsum")

llm = HuggingFacePipeline(pipeline=summarizer_pipe)

# 6. RetrievalQA Chain (acts like RAG)
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

# 7. Ask a summarization prompt
query = "Please summarize the contents of this PDF document"
response = rag_chain.invoke(query)

print("📄 Summary:\n", response['result'])


/tmp/ipython-input-3-1669660357.py:19: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public mo

📄 Summary:
 Alphabet Inc.'s annual report on Form 10-K for the year ended December 31, 2017 is available on the US Securities and Exchange Commission's website at www.sec.gov


In [4]:

# 7. Ask a summarization prompt
query = "Please summarize the contents of this PDF document"
response = rag_chain.invoke(query)

print("📄 Summary:\n", response['result'])

📄 Summary:
 Alphabet Inc.'s annual report on Form 10-K for the year ended December 31, 2017 is available on the US Securities and Exchange Commission's website at www.sec.gov
